# Week 2 Assignment: CIFAR-10 Autoencoder

For this week, you will create a convolutional autoencoder for the [CIFAR10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset. You are free to choose the architecture of your autoencoder provided that the output image has the same dimensions as the input image.

After training, your model should meet loss and accuracy requirements when evaluated with the test dataset. You will then download the model and upload it in the classroom for grading. 

Let's begin!

***Important:*** *This colab notebook has read-only access so you won't be able to save your changes. If you want to save your work periodically, please click `File -> Save a Copy in Drive` to create a copy in your account, then work from there.*  

## Imports

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [ ]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load('cifar10', as_supervised=True, split='train')

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load('cifar10', as_supervised=True, split='test')

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)

# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)

### END CODE HERE ###

## Build the Model

Create the autoencoder model. As shown in the lectures, you will want to downsample the image in the encoder layers then upsample it in the decoder path. Note that the output layer should be the same dimensions as the original image. Your input images will have the shape `(32, 32, 3)`. If you deviate from this, your model may not be recognized by the grader and may fail. 

We included a few hints to use the Sequential API below but feel free to remove it and use the Functional API just like in the ungraded labs if you're more comfortable with it. Another reason to use the latter is if you want to visualize the encoder output. As shown in the ungraded labs, it will be easier to indicate multiple outputs with the Functional API. That is not required for this assignment though so you can just stack layers sequentially if you want a simpler solution.

In [ ]:
### START CODE HERE ###

from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D
from keras import Model

def encoder(x, init_num_fil=32, num_blocks=4):
  """Return encoder model and number of filters to pass to bottleneck layer"""

  # Set initial number of filters, which will be increased as loop iterates
  num_fil = init_num_fil

  for i in range(num_blocks):
    # Define each block as convolution followed by batch norm
    x = Conv2D(filters=num_fil, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    # After every other block, add max pooling
    if i % 2 == 1:
      x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Double number of filters
    num_fil *= 2

  return x, num_fil

def bottleneck(x, num_fil):
  """Return bottleneck layer, encoder visualization, and initial number of filters for decoder model"""

  # Add convolution & batch norm bottleneck
  x = Conv2D(filters=num_fil, kernel_size=(3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)

  # Construct 2D encoder visualization
  encoder_visualization = Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same')(x)

  return x, encoder_visualization, num_fil / 2

def decoder(x, init_num_fil=256, num_blocks=4):
  """Define decoder model to reconstruct original image"""

  # Set initial number of filters, which will be decreased as loop iterates
  num_fil = init_num_fil

  for i in range(num_blocks):
    # Start with upsampling - add upsampling after every other block
    if i % 2 == 0:
      x = UpSampling2D(size=(2, 2))(x)

    # Define each block as convolution followed by batch norm
    x = Conv2D(filters=num_fil, kernel_size=(3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    
    # Halve number of filters
    num_fil /= 2
  
  # Create final reconstruction
  x = Conv2D(filters=3, kernel_size=(3, 3), activation='sigmoid', padding='same')(x)

  return x

def autoencoder(input_dim=(32, 32, 3), init_num_fil=32, num_blocks=4):
  """Create autoencoder model to recreate images"""

  # Define model components
  inputs = Input(shape=input_dim)
  encoder_output, bottleneck_num_fil = encoder(inputs, init_num_fil=init_num_fil, num_blocks=num_blocks)
  bottleneck_layer, encoder_visualization, decoder_num_fil = bottleneck(encoder_output, bottleneck_num_fil)
  decoder_output = decoder(bottleneck_layer, init_num_fil=decoder_num_fil, num_blocks=num_blocks)
  
  # Define encoder model & full autoencoder model
  encoder_model = Model(inputs=inputs, outputs=encoder_visualization)
  model = Model(inputs=inputs, outputs=decoder_output)

  return model, encoder_model

# Instantiate models
model, encoder_model = autoencoder()

### END CODE HERE ###

model.summary()

In [ ]:
encoder_model.summary()

## Configure training parameters

We have already provided the optimizer, metrics, and loss in the code below.

In [ ]:
# Please do not change the model.compile() parameters
model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')

## Training

You can now use [model.fit()](https://keras.io/api/models/model_training_apis/#fit-method) to train your model. You will pass in the `train_dataset` and you are free to configure the other parameters. As with any training, you should see the loss generally going down and the accuracy going up with each epoch. If not, please revisit the previous sections to find possible bugs.

*Note: If you get a `dataset length is infinite` error. Please check how you defined `train_dataset`. You might have included a [method that repeats the dataset indefinitely](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#repeat).*

In [ ]:
# parameters (feel free to change this)
train_steps = len(train_dataset) // BATCH_SIZE 
val_steps = len(test_dataset) // BATCH_SIZE

### START CODE HERE ###
history = model.fit(train_dataset.repeat(3),
                    epochs=len(train_dataset.repeat(3)) // train_steps,
                    steps_per_epoch=train_steps, 
                    validation_data=test_dataset, 
                    validation_steps=val_steps)
### END CODE HERE ###

## Model evaluation

You can use this code to test your model locally before uploading to the grader. To pass, your model needs to satisfy these two requirements:

* loss must be less than 0.01 
* accuracy must be greater than 0.6

In [ ]:
# Plot training losses
plt.plot(history.history['loss']);

In [ ]:
# Evaluate model on test data
result = model.evaluate(test_dataset, steps=10)

In [ ]:
def display_one_row(disp_images, offset, shape=(32, 32, 3)):
  '''Display sample outputs in one row'''

  for idx, image in enumerate(disp_images):
    # For each image in disp_images, index into proper subplot
    plt.subplot(3, 10, offset + idx + 1)
    plt.xticks([])
    plt.yticks([])

    # Display current iamge
    image = np.reshape(image, shape)
    plt.imshow(image)


def display_results(disp_input_images, disp_encoded, disp_predicted, enc_shape=(8, 8)):
  '''Display the input, encoded, and decoded output images'''

  plt.figure(figsize=(15, 5))

  # Display input images in row 1
  display_one_row(disp_input_images, 0, shape=(32,32,3))

  # Display encoder representations in row 2
  display_one_row(disp_encoded, 10, shape=enc_shape)

  # Display decoded reconstructions in row 3
  display_one_row(disp_predicted, 20, shape=(32,32,3))


In [ ]:
# Take 1 batch of the test dataset
test_sample = test_dataset.take(1)

# Put batch of images in a list
orig_images = []
for input_image, image in tfds.as_numpy(test_sample):
  orig_images = input_image

# Pick 10 random indices from list
idxs = np.random.choice(np.arange(0, len(orig_images)), size=10, replace=False)

# Get images to display
orig_sample = np.array(orig_images[idxs])
orig_sample = np.reshape(orig_sample, (10, 32, 32, 3))

# Get encoder representations
encoded = encoder_model.predict(orig_sample)

# Get decoder predictions
predicted = model.predict(orig_sample)

# Display the samples, encodings, and decoded values
display_results(orig_sample, encoded, predicted, enc_shape=(8, 8))

## Save your model

Once you are satisfied with the results, you can now save your model. Please download it from the Files window on the left and go back to the Submission portal in Coursera for grading.

In [ ]:
model.save('mymodel.h5')

**Congratulations on completing this week's assignment!**